In [2]:
import pandas as pd

In [3]:
import seaborn as sns
import matplotlib.pyplot as plt

In [17]:
from sklearn.feature_extraction import DictVectorizer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error

In [5]:
df_jan = pd.read_parquet('../data/green_tripdata_2021-01.parquet')

In [6]:
df_feb = pd.read_parquet('../data/green_tripdata_2021-02.parquet')

In [7]:
df_jan.shape, df_feb.shape

((76518, 20), (64572, 20))

In [8]:
def preprocess_data(df):
    df["trip_duration"] = df.lpep_dropoff_datetime - df.lpep_pickup_datetime
    df["trip_duration"] = df.trip_duration.apply(lambda td: td.total_seconds()/60)
    df = df[(df.trip_duration>=1) & (df.trip_duration<=60)]
    categorical = ['PULocationID', 'DOLocationID']
    df[categorical] = df[categorical].astype(str)
    return df

In [9]:
df_train = preprocess_data(df_jan)
df_val = preprocess_data(df_feb)

/tmp/ipykernel_28242/1384140220.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)
/tmp/ipykernel_28242/1384140220.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[categorical] = df[categorical].astype(str)


In [10]:
df_train.shape, df_val.shape

((73908, 21), (61921, 21))

In [21]:
df_train['PU_DO'] = df_train['PULocationID'] + "_" + df_train['DOLocationID']
df_val['PU_DO'] = df_val['PULocationID'] + "_" + df_val['DOLocationID']

/tmp/ipykernel_28242/3744494210.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_train['PU_DO'] = df_train['PULocationID'] + "_" + df_train['DOLocationID']
/tmp/ipykernel_28242/3744494210.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_val['PU_DO'] = df_val['PULocationID'] + "_" + df_val['DOLocationID']


In [22]:
categorical = ['PU_DO']
numerical = ['trip_distance']
dv = DictVectorizer()

train_dicts = df_train[categorical + numerical].to_dict(orient='records')
X_train= dv.fit_transform(train_dicts)

val_dicts = df_val[categorical + numerical].to_dict(orient='records')
X_val = dv.transform(val_dicts)

In [23]:
target = 'trip_duration'
y_train = df_train[target].values
y_val = df_val[target].values

In [24]:
lr = LinearRegression()
lr.fit(X_train, y_train)

y_pred = lr.predict(X_val)
mean_squared_error(y_val, y_pred, squared=False)

7.4793657829373155

In [28]:
import pickle


with open('../models/lin_reg.bin', 'wb') as f_out:
    pickle.dump((dv, lr), f_out)